In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, precision_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.kernel_approximation import Nystroem
import joblib

# Load the data
#uploaded = files.upload()
#file_name = list(uploaded.keys())[0]
#file_name = "spam_ham_dataset.csv"
file_name = "./data/spam_Emails_data.csv"
mail = pd.read_csv(file_name, encoding='latin1', on_bad_lines='skip')
mail


,label,text
0,Spam,viiiiiiagraaaa\nonly for the ones that want to...
1,Ham,got ice thought look az original message ice o...
2,Spam,yo ur wom an ne eds an escapenumber in ch ma n...
3,Spam,start increasing your odds of success & live s...
4,Ham,author jra date escapenumber escapenumber esca...
...,...,...
193847,Ham,on escapenumber escapenumber escapenumber rob ...
193848,Spam,we have everything you need escapelong cialesc...
193849,Ham,hi quick question say i have a date variable i...
193850,Spam,thank you for your loan request which we recie...


In [ ]:
print(mail.columns)

Index(['Unnamed: 0', 'label', 'text', 'label_num'], dtype='object')


In [ ]:
print(mail.head())


In [106]:
mail = mail.rename(columns={'label': 'Category', 'text': 'Message'})
df = mail.where(pd.notnull(mail), '')
df.loc[df['Category'] == 'Spam', 'Category'] = 1
df.loc[df['Category'] == 'Ham', 'Category'] = 0

#X-treme
df = df
#big
#df = df.groupby('Category', group_keys=False).apply(lambda x: x.sample(frac=0.324, random_state=42))
#medium
#df = df.groupby('Category', group_keys=False).apply(lambda x: x.sample(frac=0.0324, random_state=42))
#small
#df = df.groupby('Category', group_keys=False).apply(lambda x: x.sample(frac=0.00324, random_state=42))

# Split the data
x = df['Message']
y = df['Category'].astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)





155081

In [103]:
#Create logistic regression pipeline
logistic_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', LogisticRegression())
])


In [105]:
# Hyperparameter tuning using GridSearchCV for logistic regression
param_grid_lr = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [0.1, 1, 10, 30],
    'clf__solver': ['liblinear', 'lbfgs']
}

grid_search_lr = GridSearchCV(logistic_pipeline, param_grid_lr, cv=5, n_jobs=-1, verbose=1)
grid_search_lr.fit(x_train, y_train)



Fitting 5 folds for each of 16 candidates, totalling 80 fits


PicklingError: Could not pickle the task to send it to the workers.

In [107]:
# Best logistic regression model evaluation
best_model = grid_search_lr.best_estimator_
y_pred = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Model Accuracy: {accuracy}")
precision = precision_score(y_test, y_pred)
print(f"Logistic Regression Model Precision: {precision}")

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [ ]:
if len(x_train) <= 1000:
  joblib.dump(best_model, "spam_clf_model_lr_small.pkl")
elif len(x_train) <= 5100:
  joblib.dump(best_model, "spam_clf_model_lr_medium.pkl")
elif len(x_train) <= 51000:
  joblib.dump(best_model, "spam_clf_model_lr_large.pkl")
else:
  joblib.dump(best_model, "spam_clf_model_lr_extreme.pkl")

In [108]:
#NB
pipelines = {
    'MultinomialNB': Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english')),
        ('clf', MultinomialNB())
    ]),
    'BernoulliNB': Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english', binary=True)),
        ('clf', BernoulliNB())
    ])
}


In [109]:
# Train and evaluate each Naive Bayes models
#https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html
results = {}
param_grid_nb = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__alpha': [0.1, 0.5, 1, 1.5, 2]
}

for name, pipeline in pipelines.items():
    grid_search_nb = GridSearchCV(pipeline, param_grid_nb, cv=5, n_jobs=-1, verbose=1)
    grid_search_nb.fit(x_train, y_train)
    best_model_nb = grid_search_nb.best_estimator_

    if len(x_train) <= 1000:
      joblib.dump(best_model_nb, f'spam_clf_model_{name}_small.pkl')
    elif len(x_train) <= 5100:
      joblib.dump(best_model_nb, f'spam_clf_model_{name}_medium.pkl')
    elif len(x_train) <= 51000:
      joblib.dump(best_model_nb, f'spam_clf_model_{name}_large.pkl')
    else:
      joblib.dump(best_model_nb, f'spam_clf_model_{name}_extreme.pkl')

    y_pred = best_model_nb.predict(x_test)

    precision = precision_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    results[name] = {'Precision': precision, 'Accuracy': accuracy}


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits


PicklingError: Could not pickle the task to send it to the workers.

In [ ]:
x_train

36697     return - path :\nreceived : from rly - yao 2 ....
116460    numerous studies of free play time have shown ...
50844     i ' ve been searching for a place to buy med `...
82514     war cold that place friend hour fill story peo...
129244    bull market report lookup alvncurrent escapenu...
                                ...                        
150258    take look note address capacity release issues...
108046    start date : 1 / 20 / 02 ; hourahead hour : 16...
119777    fyi .\ntj - - will you please forward to kevin...
157100    alternatively generate the log likelihood usin...
49315     the best source for d rugs anywhere no fees st...
Name: Message, Length: 5024, dtype: object

In [ ]:
y_train

In [ ]:
for model, metrics in results.items():
    print(f"{model} Model Precision: {metrics['Precision']}")
    print(f"{model} Model Accuracy: {metrics['Accuracy']}")

MultinomialNB Model Precision: 0.9917582417582418
MultinomialNB Model Accuracy: 0.9778697659608343
BernoulliNB Model Precision: 0.8507804668480595
BernoulliNB Model Accuracy: 0.9118770896354084


In [ ]:
random_forest_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', RandomForestClassifier(random_state=42))
])

param_grid_rf = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__max_depth': [50, 75 , 100, 150, 200, 300],
    'clf__min_impurity_decrease': [0.0001],
}

grid_search_rf = GridSearchCV(random_forest_pipeline, param_grid_rf, cv=5, n_jobs=-1, verbose=1)
grid_search_rf.fit(x_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words='english')),
                                       ('clf',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'clf__max_depth': [50, 75, 100, 150, 200, 300],
                         'clf__min_impurity_decrease': [0.0001],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             verbose=1)

In [54]:
best_model_rf.get_params()

{'memory': None,
 'steps': [('tfidf', TfidfVectorizer(stop_words='english')),
  ('clf',
   RandomForestClassifier(max_depth=150, min_impurity_decrease=0.0001,
                          random_state=42))],
 'verbose': False,
 'tfidf': TfidfVectorizer(stop_words='english'),
 'clf': RandomForestClassifier(max_depth=150, min_impurity_decrease=0.0001,
                        random_state=42),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': None,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 1),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': 'english',
 'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': False,
 'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__tokenizer': None,
 'tfidf__use_idf': True,
 'tfidf__vocabulary': None,


In [ ]:
# Best Random Forest model evaluation
best_model_rf = grid_search_rf.best_estimator_

if len(x_train) <= 1000:
    joblib.dump(best_model_rf, 'spam_clf_model_rf_small.pkl')
elif len(x_train) <= 5100:
    joblib.dump(best_model_rf, 'spam_clf_model_rf_medium.pkl')
elif len(x_train) <= 51000:
    joblib.dump(best_model_rf, 'spam_clf_model_rf_large.pkl')
else:
    joblib.dump(best_model_rf, 'spam_clf_model_rf_extreme.pkl')

y_pred = best_model_rf.predict(x_test)

print(f"Random Forest Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Random Forest Precision {precision_score(y_test, y_pred)}")

Random Forest Accuracy: 0.9495303295653559
Random Forest Precision 0.9491638795986622


In [ ]:
svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', SVC(random_state=42))
])

param_grid_svm = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [2, 3, 5]
}

grid_search_svm = GridSearchCV(svm_pipeline, param_grid_svm, cv=5, n_jobs=-1, verbose=1)
grid_search_svm.fit(x_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words='english')),
                                       ('clf', SVC(random_state=42))]),
             n_jobs=-1,
             param_grid={'clf__C': [2, 3, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             verbose=1)

In [90]:
best_model_svm = grid_search_svm.best_estimator_

y_pred = best_model_svm.predict(x_test)

print(f"SVM Model Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"SVM Model Precision {precision_score(y_test, y_pred)}")

if len(x_train) <= 1000:
    joblib.dump(best_model_svm, 'spam_clf_model_svm_small.pkl')
elif len(x_train) <= 5100:
    joblib.dump(best_model_svm, 'spam_clf_model_svm_medium.pkl')
elif len(x_train) <= 51000:
    joblib.dump(best_model_svm, 'spam_clf_model_svm_large.pkl')
else:
    joblib.dump(best_model_svm, 'spam_clf_model_svm_extreme.pkl')



SVM Model Accuracy: 0.9196499602227526
SVM Model Precision 0.9022435897435898


In [ ]:
best_model_svm.get_params()

{'memory': None,
 'steps': [('tfidf', TfidfVectorizer(stop_words='english')),
  ('clf', SVC(C=3))],
 'verbose': False,
 'tfidf': TfidfVectorizer(stop_words='english'),
 'clf': SVC(C=3),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': None,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 1),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': 'english',
 'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': False,
 'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__tokenizer': None,
 'tfidf__use_idf': True,
 'tfidf__vocabulary': None,
 'clf__C': 3,
 'clf__break_ties': False,
 'clf__cache_size': 200,
 'clf__class_weight': None,
 'clf__coef0': 0.0,
 'clf__decision_function_shape': 'ovr',
 'clf__degree': 3,
 'clf__gamma': 'scale',
 'clf__k